# Keras Exercise

## Predict political party based on votes

As a fun little example, we'll use a public data set of how US congressmen voted on 17 different issues in the year 1984. Let's see if we can figure out their political party based on their votes alone, using a deep neural network!

For those outside the United States, our two main political parties are "Democrat" and "Republican." In modern times they represent progressive and conservative ideologies, respectively.

Politics in 1984 weren't quite as polarized as they are today, but you should still be able to get over 90% accuracy without much trouble.

Since the point of this exercise is implementing neural networks in Keras, I'll help you to load and prepare the data.

Let's start by importing the raw CSV file using Pandas, and make a DataFrame out of it with nice column labels:

In [10]:
import pandas as pd

feature_names =  ['party','handicapped-infants', 'water-project-cost-sharing', 
                    'adoption-of-the-budget-resolution', 'physician-fee-freeze',
                    'el-salvador-aid', 'religious-groups-in-schools',
                    'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
                    'mx-missle', 'immigration', 'synfuels-corporation-cutback',
                    'education-spending', 'superfund-right-to-sue', 'crime',
                    'duty-free-exports', 'export-administration-act-south-africa']

voting_data = pd.read_csv('house-votes-84.data.txt', na_values=['?'], 
                          names = feature_names)
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


We can use describe() to get a feel of how the data looks in aggregate:

In [11]:
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


We can see there's some missing data to deal with here; some politicians abstained on some votes, or just weren't present when the vote was taken. We will just drop the rows with missing data to keep it simple, but in practice you'd want to first make sure that doing so didn't introduce any sort of bias into your analysis (if one party abstains more than another, that could be problematic for example.)

In [12]:
voting_data.dropna(inplace=True)
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


Our neural network needs normalized numbers, not strings, to work. So let's replace all the y's and n's with 1's and 0's, and represent the parties as 1's and 0's as well.

In [13]:
voting_data.replace(('y', 'n'), (1, 0), inplace=True)
voting_data.replace(('democrat', 'republican'), (1, 0), inplace=True)

In [5]:
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missle,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
5,1,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,1,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,1,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


Finally let's extract the features and labels in the form that Keras will expect:

In [17]:
all_features = voting_data[feature_names].drop('party', axis=1).values
all_classes = voting_data['party'].values

OK, so have a go at it! You'll want to refer back to the slide on using Keras with binary classification - there are only two parties, so this is a binary problem. This also saves us the hassle of representing classes with "one-hot" format like we had to do with MNIST; our output is just a single 0 or 1 value.

Also refer to the scikit_learn integration slide, and use cross_val_score to evaluate your resulting model with 10-fold cross-validation.

**If you're using tensorflow-gpu on a Windows machine** by the way, you probably *do* want to peek a little bit at my solution - if you run into memory allocation errors, there's a workaround there you can use.

Try out your code here:

In [23]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

def create_model():
    model = Sequential()
    #16 feature inputs (votes) going into an 32-unit layer 
    model.add(Dense(8, input_dim=16, kernel_initializer='normal', activation='sigmoid'))
    # Another hidden layer of 16 units
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    # Another hidden layer of 16 units
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    # Output layer with a binary classification (Democrat or Republican political party)
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=1)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)
cv_scores.mean()

Epoch 1/100
208/208 [==============================] - 0s 441us/sample - loss: 0.6931 - accuracy: 0.5240
Epoch 2/100
208/208 [==============================] - 0s 54us/sample - loss: 0.6931 - accuracy: 0.5240
Epoch 3/100
208/208 [==============================] - 0s 28us/sample - loss: 0.6929 - accuracy: 0.5240
Epoch 4/100
208/208 [==============================] - 0s 71us/sample - loss: 0.6928 - accuracy: 0.5240
Epoch 5/100
208/208 [==============================] - 0s 88us/sample - loss: 0.6926 - accuracy: 0.5240
Epoch 6/100
208/208 [==============================] - 0s 82us/sample - loss: 0.6923 - accuracy: 0.5240
Epoch 7/100
208/208 [==============================] - 0s 80us/sample - loss: 0.6919 - accuracy: 0.5240
Epoch 8/100
208/208 [==============================] - 0s 82us/sample - loss: 0.6912 - accuracy: 0.5240
Epoch 9/100
208/208 [==============================] - 0s 80us/sample - loss: 0.6901 - accuracy: 0.5240
Epoch 10/100
208/208 [==============================] - 0s 112u

208/208 [==============================] - 0s 101us/sample - loss: 0.0865 - accuracy: 0.9712
Epoch 79/100
208/208 [==============================] - 0s 83us/sample - loss: 0.0831 - accuracy: 0.9760
Epoch 80/100
208/208 [==============================] - 0s 102us/sample - loss: 0.0816 - accuracy: 0.9760
Epoch 81/100
208/208 [==============================] - 0s 80us/sample - loss: 0.0801 - accuracy: 0.9760
Epoch 82/100
208/208 [==============================] - 0s 77us/sample - loss: 0.0801 - accuracy: 0.9760
Epoch 83/100
208/208 [==============================] - 0s 79us/sample - loss: 0.0796 - accuracy: 0.9760
Epoch 84/100
208/208 [==============================] - 0s 80us/sample - loss: 0.0754 - accuracy: 0.9712
Epoch 85/100
208/208 [==============================] - 0s 79us/sample - loss: 0.0742 - accuracy: 0.9760
Epoch 86/100
208/208 [==============================] - 0s 79us/sample - loss: 0.0729 - accuracy: 0.9760
Epoch 87/100
208/208 [==============================] - 0s 73us/sa

208/208 [==============================] - 0s 64us/sample - loss: 0.1729 - accuracy: 0.9471
Epoch 56/100
208/208 [==============================] - 0s 63us/sample - loss: 0.1679 - accuracy: 0.9471
Epoch 57/100
208/208 [==============================] - 0s 68us/sample - loss: 0.1633 - accuracy: 0.9471
Epoch 58/100
208/208 [==============================] - 0s 71us/sample - loss: 0.1591 - accuracy: 0.9471
Epoch 59/100
208/208 [==============================] - 0s 63us/sample - loss: 0.1542 - accuracy: 0.9471
Epoch 60/100
208/208 [==============================] - 0s 59us/sample - loss: 0.1505 - accuracy: 0.9471
Epoch 61/100
208/208 [==============================] - 0s 55us/sample - loss: 0.1469 - accuracy: 0.9519
Epoch 62/100
208/208 [==============================] - 0s 55us/sample - loss: 0.1428 - accuracy: 0.9471
Epoch 63/100
208/208 [==============================] - 0s 63us/sample - loss: 0.1395 - accuracy: 0.9519
Epoch 64/100
208/208 [==============================] - 0s 67us/samp

209/209 [==============================] - 0s 62us/sample - loss: 0.2805 - accuracy: 0.9091
Epoch 33/100
209/209 [==============================] - 0s 60us/sample - loss: 0.2683 - accuracy: 0.9139
Epoch 34/100
209/209 [==============================] - 0s 60us/sample - loss: 0.2570 - accuracy: 0.9139
Epoch 35/100
209/209 [==============================] - 0s 54us/sample - loss: 0.2500 - accuracy: 0.9091
Epoch 36/100
209/209 [==============================] - 0s 55us/sample - loss: 0.2395 - accuracy: 0.9187
Epoch 37/100
209/209 [==============================] - 0s 59us/sample - loss: 0.2331 - accuracy: 0.9234
Epoch 38/100
209/209 [==============================] - 0s 68us/sample - loss: 0.2261 - accuracy: 0.9234
Epoch 39/100
209/209 [==============================] - 0s 66us/sample - loss: 0.2211 - accuracy: 0.9187
Epoch 40/100
209/209 [==============================] - 0s 69us/sample - loss: 0.2154 - accuracy: 0.9234
Epoch 41/100
209/209 [==============================] - 0s 72us/samp

209/209 [==============================] - 0s 52us/sample - loss: 0.6909 - accuracy: 0.5263
Epoch 10/100
209/209 [==============================] - 0s 49us/sample - loss: 0.6896 - accuracy: 0.5981
Epoch 11/100
209/209 [==============================] - 0s 57us/sample - loss: 0.6876 - accuracy: 0.8660
Epoch 12/100
209/209 [==============================] - 0s 65us/sample - loss: 0.6846 - accuracy: 0.9043
Epoch 13/100
209/209 [==============================] - 0s 65us/sample - loss: 0.6804 - accuracy: 0.9139
Epoch 14/100
209/209 [==============================] - 0s 62us/sample - loss: 0.6745 - accuracy: 0.8852
Epoch 15/100
209/209 [==============================] - 0s 46us/sample - loss: 0.6656 - accuracy: 0.8852
Epoch 16/100
209/209 [==============================] - 0s 52us/sample - loss: 0.6535 - accuracy: 0.8852
Epoch 17/100
209/209 [==============================] - 0s 62us/sample - loss: 0.6390 - accuracy: 0.8517
Epoch 18/100
209/209 [==============================] - 0s 64us/samp

...

209/209 [==============================] - 0s 61us/sample - loss: 0.1700 - accuracy: 0.9378
Epoch 97/100
209/209 [==============================] - 0s 65us/sample - loss: 0.1663 - accuracy: 0.9378
Epoch 98/100
209/209 [==============================] - 0s 59us/sample - loss: 0.1628 - accuracy: 0.9330
Epoch 99/100
209/209 [==============================] - 0s 68us/sample - loss: 0.1596 - accuracy: 0.9330
Epoch 100/100
23/23 [==============================] - 0s 1ms/sample - loss: 0.1080 - accuracy: 1.0000
Epoch 1/100
209/209 [==============================] - 0s 443us/sample - loss: 0.6931 - accuracy: 0.5407
Epoch 2/100
209/209 [==============================] - 0s 34us/sample - loss: 0.6929 - accuracy: 0.5407
Epoch 3/100
209/209 [==============================] - 0s 30us/sample - loss: 0.6926 - accuracy: 0.5407
Epoch 4/100
209/209 [==============================] - 0s 53us/sample - loss: 0.6924 - accuracy: 0.5407
Epoch 5/100
209/209 [==============================] - 0s 34us/sample - l

209/209 [==============================] - 0s 62us/sample - loss: 0.1143 - accuracy: 0.9665
Epoch 74/100
209/209 [==============================] - 0s 77us/sample - loss: 0.1116 - accuracy: 0.9665
Epoch 75/100
209/209 [==============================] - 0s 82us/sample - loss: 0.1091 - accuracy: 0.9665
Epoch 76/100
209/209 [==============================] - 0s 68us/sample - loss: 0.1063 - accuracy: 0.9665
Epoch 77/100
209/209 [==============================] - 0s 87us/sample - loss: 0.1042 - accuracy: 0.9665
Epoch 78/100
209/209 [==============================] - 0s 63us/sample - loss: 0.1020 - accuracy: 0.9713
Epoch 79/100
209/209 [==============================] - 0s 62us/sample - loss: 0.1008 - accuracy: 0.9713
Epoch 80/100
209/209 [==============================] - 0s 67us/sample - loss: 0.0977 - accuracy: 0.9713
Epoch 81/100
209/209 [==============================] - 0s 63us/sample - loss: 0.0954 - accuracy: 0.9713
Epoch 82/100
209/209 [==============================] - 0s 68us/samp

209/209 [==============================] - 0s 60us/sample - loss: 0.1648 - accuracy: 0.9426
Epoch 51/100
209/209 [==============================] - 0s 72us/sample - loss: 0.1609 - accuracy: 0.9426
Epoch 52/100
209/209 [==============================] - 0s 71us/sample - loss: 0.1582 - accuracy: 0.9474
Epoch 53/100
209/209 [==============================] - 0s 77us/sample - loss: 0.1547 - accuracy: 0.9522
Epoch 54/100
209/209 [==============================] - 0s 77us/sample - loss: 0.1512 - accuracy: 0.9522
Epoch 55/100
209/209 [==============================] - 0s 75us/sample - loss: 0.1477 - accuracy: 0.9522
Epoch 56/100
209/209 [==============================] - 0s 68us/sample - loss: 0.1442 - accuracy: 0.9522
Epoch 57/100
209/209 [==============================] - 0s 63us/sample - loss: 0.1414 - accuracy: 0.9522
Epoch 58/100
209/209 [==============================] - 0s 75us/sample - loss: 0.1386 - accuracy: 0.9522
Epoch 59/100
209/209 [==============================] - 0s 73us/samp

209/209 [==============================] - 0s 88us/sample - loss: 0.4528 - accuracy: 0.8756
Epoch 28/100
209/209 [==============================] - 0s 66us/sample - loss: 0.4467 - accuracy: 0.8756
Epoch 29/100
209/209 [==============================] - 0s 67us/sample - loss: 0.4414 - accuracy: 0.8852
Epoch 30/100
209/209 [==============================] - 0s 63us/sample - loss: 0.4362 - accuracy: 0.8804
Epoch 31/100
209/209 [==============================] - 0s 60us/sample - loss: 0.4312 - accuracy: 0.8852
Epoch 32/100
209/209 [==============================] - 0s 63us/sample - loss: 0.4266 - accuracy: 0.8852
Epoch 33/100
209/209 [==============================] - 0s 65us/sample - loss: 0.4221 - accuracy: 0.8900
Epoch 34/100
209/209 [==============================] - 0s 68us/sample - loss: 0.4180 - accuracy: 0.8900
Epoch 35/100
209/209 [==============================] - 0s 68us/sample - loss: 0.4136 - accuracy: 0.8900
Epoch 36/100
209/209 [==============================] - 0s 72us/samp

0.9483695685863495

## My implementation is below

# No peeking!

![title](peek.jpg)

In [19]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

def create_model():
    model = Sequential()
    #16 feature inputs (votes) going into an 32-unit layer 
    model.add(Dense(32, input_dim=16, kernel_initializer='normal', activation='relu'))
    # Another hidden layer of 16 units
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    # Output layer with a binary classification (Democrat or Republican political party)
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, all_features, all_classes, cv=10)
cv_scores.mean()

0.9438405811786652

94% without even trying too hard! Did you do better? Maybe more neurons, more layers, or Dropout layers would help even more.